In [ ]:
from panel_segmentation.lidar import pcd_data, plane_segmentation, usgs_lidar_api
from shapely.wkt import loads
import os
import open3d as o3d
import plotly.graph_objects as go
import numpy as np
import pandas as pd

Get a list of shapely polygons to run LiDAR pipeline. Polygons can be generated from inferences

In [3]:
# Shapely polygon of a panel
shapely_polygon = loads("""POLYGON ((
                        -97.74106315632842 30.14782268767431,
                        -97.74096919534045 30.1478079931521,
                        -97.74096768230025 30.1478196557987,
                        -97.74099649481839 30.14782495927133,
                        -97.74099415225778 30.1478379364854,
                        -97.74103533818115 30.14784324186061,
                        -97.74103979247948 30.14783185545246,
                        -97.74105752041777 30.14783384726395,
                        -97.74106315632842 30.14782268767431))""")
# Get.laz file that contains the shapely polygon coordinates
laz_file_path = os.path.join(os.getcwd(), r"panel_segmentation\examples\lidar_examples\cropped_USGS_LPC_TX_Central_B1_2017_stratmap17_50cm_3097513c1_LAS_2019.laz")
print(laz_file_path)


c:\Users\Quyen\Documents\Panel-Segmentation\panel_segmentation\examples\lidar_examples\cropped_USGS_LPC_TX_Central_B1_2017_stratmap17_50cm_3097513c1_LAS_2019.laz


Visualizing point cloud data can be easily done with [o3d.visualization.draw_geometries()](https://www.open3d.org/docs/latest/python_api/open3d.visualization.draw_geometries.html) function. However, this opens a new browser with the displayed figure. To keep the figures inline with the Juypter notebook and avoid pop-up browsers, the following example will use plotly to plot the pcd data interactively.

In [ ]:
# Initialize pcd class
pcd = pcd_data.PCD(laz_file_path, polygon=shapely_polygon)
# Read, crop, and filter pcd data to only contain the points inside the polygon
filtered_pcd = pcd.readCropLaz()
# Preprocess pcd
preprocessed_pcd = pcd.preprocessPcd(filtered_pcd)

# Visualize preprocessed pcd with plotly
# First, convert open3d pcd to numpy array
points = np.asarray(preprocessed_pcd.points)
# Create a 3d scatter plot with the points
fig = go.Figure(
    data=[go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], mode="markers")],
    layout=go.Layout(
        title="Filtered and Preprocessed PCD",
        width=1200, height=1000,
        scene=dict(
            xaxis_title="X (meters)", yaxis_title="Y (meters)", zaxis_title="Z (meters)",
            # This controls the plot view and zoom. Smaller x, y, z values zooms in.
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)))
        ))
fig.show()

In [92]:
# Initialize PanelSegmentation class with preprocessed pcd
ps = plane_segmentation.PlaneSegmentation(preprocessed_pcd)
# Segment lidar points into planes
ps.segmentPlanes()

# Create plane mesh for visualization 
plane_mesh_list = ps.visualizePlanes()



